# MyoSDK Tutorial

This notebook demonstrates how to use `myosdk` to process a video, extract motion (tracking), and retarget that motion onto a 3D character.

In [ ]:
import os
from myosdk import Client
from rich import print

# Initialize the client
# You can set MYOSDK_API_KEY in your environment variables or pass it directly.
# Here we check for the env var, and default to a hardcoded key for demonstration if missing.
key = os.getenv("MYOSDK_API_KEY", "...")
client = Client(api_key=key, base_url="https://v2m-alb-us-east-1.myolab.ai")

print("Client initialized successfully.")

## 1. Explore Available Characters

Before we start, let's see what characters are available for retargeting.

In [ ]:
characters = client.characters.list(limit=5)
print(characters)

## 2. Upload Video

We need to upload a video file to the platform. This video will be used to extract motion.

In [ ]:
video_path = "/home/ouassim/dev/myoGalleryKinesis/tmp-assets/input.mp4"

print(f"Uploading {video_path}...")
input_asset = client.assets.upload_file(video_path)
print("Video uploaded:", input_asset)

## 3. Motion Tracking

Now we start a tracker job to extract motion data from the uploaded video.

In [ ]:
print("Starting tracker job...")
tracker_job = client.jobs.start_tracker(
    video_asset_id=input_asset["asset_id"],
    params={"fps": 30, "skeleton": "humanoid"}
)
print("Tracker job started:", tracker_job["job_id"])

In [ ]:
print("Waiting for tracker job to complete...")
tracker_result = client.jobs.wait(tracker_job["job_id"])

if tracker_result["status"] == "SUCCEEDED":
    trackers_asset_id = tracker_result["output"]["trackers_asset_id"]
    print(f"Tracker job succeeded. Output Asset ID: {trackers_asset_id}")
else:
    print("Tracker job failed:", tracker_result)

## 4. Motion Retargeting

Use the extracted motion data (trackers) to retarget onto a 3D character.

In [ ]:
# Using a specific character ID (replace with one from the list above if needed)
character_id = "0199e278-06e6-7726-a6ba-9929f79005e8"
character_version = "v1.0.0"

print(f"Starting retargeting job for character {character_id}...")
retarget_job = client.jobs.start_retarget(
    tracker_asset_id=trackers_asset_id,
    character_id=character_id,
    character_version=character_version
)
print("Retarget job started:", retarget_job["job_id"])

In [ ]:
print("Waiting for retarget job to complete...")
retarget_result = client.jobs.wait(retarget_job["job_id"])

if retarget_result["status"] == "SUCCEEDED":
    motion_asset_id = retarget_result["output"]["motion_asset_id"]
    print(f"Retarget job succeeded. Output Asset ID: {motion_asset_id}")
else:
    print("Retarget job failed:", retarget_result)

## 5. Download Result

Finally, download the generated GLB file.

In [ ]:
output_path = "tmp-assets/tutorial_output.glb"

print(f"Downloading result to {output_path}...")
client.assets.download(motion_asset_id, output_path)
print("Download complete!")